<a href="https://colab.research.google.com/github/worldhello23/nn/blob/main/f_rag_voyager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
pine = userdata.get('pinecone_api')
tog = userdata.get('together')
voy = userdata.get('voyager')

In [2]:
!pip config --user set global.progress_bar off
%pip install -qqq llama-index-vector-stores-pinecone
!pip install -qqq sentence-transformers
!pip install -qqq pydantic==1.10.11 pinecone-client>=3.0.0
%pip install llama-index-embeddings-together
!pip install -U voyageai

Writing to /root/.config/pip/pip.conf


In [4]:
import pandas as pd
import numpy as np

In [5]:
df_test = pd.read_csv("/content/all_questions_test_public.csv")
df_test.head()

,question,ID
0,What factors make Actinium Pharmaceuticals (AT...,0
1,What potential impact could Vertex Pharmaceuti...,1
2,What potential impact could Actinium Pharmaceu...,2
3,How does the integration of cryptocurrency wal...,3
4,Question: How can individuals utilize decentra...,4


In [6]:
df_test_a = pd.read_csv("/content/all_answers_test_public.csv")
df_test_a.head()

,raw_text,answer_id
0,"COLLEGE PARK, Md., April 05, 2024--(BUSINESS W...",0
1,Wirestock\n\nNo matter the positive projection...,1
2,Loading... Loading...\n\nClinical-stage psyche...,2
3,"In the phase IIb expansion, patients will be r...",3
4,\n\n\n\n\n\nSequans to Showcase its Cellular I...,4


# pinecone

In [7]:
from pinecone import Pinecone
pc = Pinecone(api_key = pine)
pinecone_index = pc.Index("fin")

In [ ]:
# pinecone_index.delete(delete_all=True, namespace='ns1')

{}

In [ ]:
# pinecone_index.upsert(pl[1000:], namespace = "ns1")

{'upserted_count': 233}

In [ ]:
l = []
for i in range(len(embeddings_q)):
  l.append(int(pinecone_index.query(
    namespace="ns1",
    vector=embeddings_q[i],
    top_k=1,
     include_metadata=True
).matches[0]['id']))

In [ ]:
pd.Series(l).to_csv("result.csv")

In [ ]:
pd.Series(l).to_csv("result.csv")

In [ ]:
pd.read_csv("result.csv")

,Unnamed: 0,0
0,0,1112
1,1,1047
2,2,363
3,3,731
4,4,328
...,...,...
1228,1228,923
1229,1229,1057
1230,1230,625
1231,1231,973


#voyager


In [8]:
import voyageai
vo = voyageai.Client(api_key = voy)

In [10]:
doc_a = df_test_a.raw_text.to_list()
doc_q = df_test.question.to_list()
e_a = pd.read_csv("series_a.csv").iloc[:, 1:]
e_q =  pd.read_csv("series_q.csv").iloc[:, 1:]
e_q = pd.Series(e_q["0"]).tolist()
e_a = pd.Series(e_a["0"]).tolist()

answer embedding


In [ ]:
# Answers
batch_size = 50
embeddings = []

for i in range(0, len(doc_a), batch_size):
    embeddings += vo.embed(
        doc_a[i:i + batch_size], model="voyage-large-2-instruct", input_type="document"
    ).embeddings

In [ ]:
# pd.Series(embeddings_q).to_csv('series_q.csv')

question embedding

In [ ]:
# Questions
embeddings_q = []

for i in range(0, len(doc_q)):
    embeddings_q += vo.embed(
        doc_q[i], model="voyage-large-2-instruct", input_type="document"
    ).embeddings

json

In [13]:
pl = []
for i in range(len(e_a)):

  pl.append( {
       "id": str(i),
        "values": e_a[i],
        "metadata" : { "text" : doc_a[i]}

  })

In [ ]:
# r = []
# for i in range(len(embeddings_q)):
#   r.append(int(pinecone_index.query(
#     namespace="ns1",
#     vector=embeddings_q[i],
#     top_k=1,
#      include_metadata=True
# ).matches[0]['id']))

#Rerankers

In [17]:
r = []
for i in range(len(e_q)):
  r.append(pinecone_index.query(
      namespace="ns1",
      vector=eval(e_q[i]),
      top_k=10,
      include_metadata=True ))
  # ).matches[i].metadata['text'])

In [36]:
r

{'matches': [{'id': '1112',
              'metadata': {'text': 'Biotech Stocks Taking Centerstage\n'
                                   '\n'
                                   'As the pharmaceutical industry ventures '
                                   'deeper into the era of precision medicine, '
                                   'radiopharmaceuticals, particularly those '
                                   'involving alpha emitters like Actinium-225 '
                                   '(Ac-225), are gaining prominence for their '
                                   'targeted approach to cancer treatment. '
                                   'Amid this innovative surge, Actinium '
                                   'Pharmaceuticals (NYSE:ATNM) emerges not '
                                   'just as a company with a promising '
                                   'pipeline but as a beacon of cutting-edge '
                                   'manufacturing capabilities that could ver

In [ ]:
# q = []
# for i in range(10):
#   q.append(r[0].matches[i].metadata['text'])
#q[8]

'Loading... Loading...\n\n\n\n"Biotech Stocks In Focus"\n\nDr. Kumar will discuss Anixa\'s portfolio of first-in-class oncology/immunology assets under development, with vaccines to prevent cancer and a CAR-T cell therapy to treat cancer. The Company\'s robust pipeline over the next 12 months includes three candidates based on two modalities, three indications, and two ongoing clinical trials.\n\n2023 witnessed a remarkable flurry of activity in the biopharmaceutical industry, with several high-profile acquisitions underscoring the sector\'s dynamic nature and its relentless pursuit of innovation. Here’s a look at some of the most significant deals that not only made headlines but are set to reshape the therapeutic landscape.\n\nBefore we get into the recent M and A’s, one biotech company that we would like to draw your attention to is Anixa Biosciences ANIX. Shares of Anixa Biosciences ANIX are up over 15% in the last 5 days.\n\nAmid the wave of acquisitions sweeping through the bioph

In [ ]:
# ls = []
for j in range(1233,1234):
  q = []
  for i in range(10):
    q.append(r[j].matches[i].metadata['text'])
  reranking = vo.rerank(doc_q[j], q, model="rerank-lite-1", top_k=1)
  doc = reranking.results[0].document


IndexError: list index out of range

In [42]:
q = []
for j in range(len(r)):

  for i in range(10):
    q.append(r[j].matches[i].metadata['text'])


In [57]:
ls = []

In [46]:
import time

In [62]:
for i in range(len(r)):
    try:
      reranking = vo.rerank(doc_q[i] ,q[i*10 :(i + 1) *10], model="rerank-lite-1", top_k=1)
      doc = reranking.results[0].document

  # Use a generator expression for efficient matching
      # matching_id = next((match['id'] for match in r[i]['matches']
      #                                 if match['metadata']['text'].strip() == doc), None)

      # ls.append(matching_id)
      for match in r[i]['matches']:
        if match['metadata']['text'].strip() == doc:
          ls.append(match['id'])

    except Exception as e:
      print(f"Rate limit hit at i={i}. Waiting and retrying...")
      time.sleep(60)  # Wait for 60 seconds before retrying
      # i -= 1

Rate limit hit at i=100. Waiting and retrying...
Rate limit hit at i=201. Waiting and retrying...
Rate limit hit at i=302. Waiting and retrying...
Rate limit hit at i=403. Waiting and retrying...
Rate limit hit at i=504. Waiting and retrying...
Rate limit hit at i=605. Waiting and retrying...
Rate limit hit at i=706. Waiting and retrying...
Rate limit hit at i=807. Waiting and retrying...
Rate limit hit at i=908. Waiting and retrying...
Rate limit hit at i=1009. Waiting and retrying...
Rate limit hit at i=1110. Waiting and retrying...
Rate limit hit at i=1211. Waiting and retrying...


In [61]:
(ls)[:25]

['1112',
 '1047',
 '363',
 '731',
 '328',
 '1221',
 '302',
 '882',
 '843',
 '1107',
 '856',
 '755',
 '497',
 '213',
 '1013',
 '248',
 '115',
 '95',
 '975',
 '982',
 '169',
 '929',
 None,
 '1086',
 '433']